In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import re
from konlpy.tag import Okt
import matplotlib.pyplot as plt

In [ ]:
# CSV 파일 읽기
csv_filename = '공지사항.csv'
df = pd.read_csv(csv_filename)

# 제목 추출 및 저장
notice_names = df['title'].tolist()
print("데이터 개수 :" , len(notice_names))
for notice in notice_names:
  print(notice)

In [ ]:
# 전처리 함수 정의
def preprocess_text(text):
    # 특수문자 제거
    text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)

    # 숫자 정보 제거
    text = re.sub(r'\d+', '', text)

    # 형태소 분석
    tagger = Okt()
    words = tagger.morphs(text)

    # 불용어 제거
    stop_words = [ '필독', '학기', '학년', '도', '년', '제', '회', '월', '학부', '일', '차', '년도', '안내']  # 불용어 리스트
    words = [word for word in words if word not in stop_words]

    # 분석된 형태소들을 공백으로 결합하여 문장으로 반환
    text = ' '.join(words)

    return text

In [ ]:
# 전처리된 공지이름 추출
preprocessed_notice_names = [preprocess_text(notice_name) for notice_name in notice_names]

print("전처리 공지 제목")

for notice in preprocessed_notice_names:
  print(notice)

In [ ]:
# GPT 모델 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

In [ ]:
# 공지사항 텍스트를 GPT 모델에 입력하여 임베딩 생성
embeddings = []
for notice in preprocessed_notice_name:
    inputs = tokenizer.encode(notice, return_tensors='pt')
    outputs = model(inputs)[0][:, 0, :].detach().numpy()
    embeddings.append(outputs)

# 임베딩을 numpy 배열로 변환
embeddings = np.vstack(embeddings)

In [ ]:
# Elbow Method를 사용하여 적절한 클러스터 개수 k 결정
inertias = []
k_values = range(200, 300)
for k in k_values:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
    kmeans.fit(embeddings)
    inertias.append(kmeans.inertia_)

print("여기까지 잘됨")

# Elbow Method를 시각화하여 적절한 k 결정
plt.plot(k_values, inertias, 'bx-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

In [ ]:
# 사용자로부터 적절한 클러스터 개수 k 입력
k = int(input("Enter the number of clusters (k): "))


# K-Means 군집화 적용
kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
kmeans.fit(embeddings)

In [ ]:
# 군집화 결과 출력
for notice, label in zip(notices, kmeans.labels_):
    print("공지사항:", notice)
    print("군집 레이블:", label)
    print()